In [1]:
import os
import sys
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import gc
import csv

### Книги

In [2]:
source = 'http://studyenglishwords.com'

In [ ]:
page = BeautifulSoup(requests.get(source+'/result/books').text, "html5lib")

In [ ]:
books = page.find("table").find_all("a")

In [ ]:
# русско- и франкоязычные писатели
irrelevant = [0,1,9,32,44,45,53,54,56,66,70,78,84,85,86,91,92,94,96,98,99]

In [ ]:
book_link = {}
for i in range(len(books)):
    if i not in irrelevant and i > 80:
        book_link[books[i].text] = books[i].get('href')

In [3]:
def replace(string, substitutions):

    substrings = sorted(substitutions, key=len, reverse=True)
    regex = re.compile('|'.join(map(re.escape, substrings)))
    return regex.sub(lambda match: substitutions[match.group(0)], string)

In [4]:
# извлекаем со страницы данные и сохраняем в df через список
def save_page(page, df, substitution):
    en = [replace(sent.find("span").text, substitutions) for sent in page.find_all("div", {"class":"left"})]
    ru = [replace(sent.find("span").text, substitutions) for sent in bookpage.find_all("div", {"class":"right"})]
    start = len(df)
    for i in range(len(en)):
        df.loc[start+i] = [en[i], ru[i]]

In [11]:
columns = ['en', 'ru']
substitutions = {'\n':' ', '\t':' '}

In [ ]:
for book in book_link:
    # открыть книгу
    bookpage = BeautifulSoup(requests.get(source+book_link[book]).text, "html5lib")
    df = pd.DataFrame(columns=columns)
    # пока есть страницы
    while bookpage:
        # сохраняем их в df
        save_page(bookpage, df)
        # получаем следующую страницу
        next_page = bookpage.find("span", {"class":"next"}).find("a")
        if next_page:
            bookpage = BeautifulSoup(requests.get(source+book_link[book]+next_page.get("href")).text, "html5lib")
        else:
            bookpage = False
    # сохранить датафрейм
    df.to_csv(book+'.tsv', sep = '\t')
    print(book+' is saved')    
    # чтобы точно очистить память от датафрейма
    del df
    gc.collect()

### Субтитры
сразу токенизируем

In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'

[nltk_data] Downloading package stopwords to /home/lisa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def tokenize(text):
    
    words = [word.strip(punct) for word in text.split() if word != '']
    words = [word for word in words if word]

    return words

In [7]:
# на двух страницах, проще вручную
page_sub1 = BeautifulSoup(requests.get('https://studyenglishwords.com/browse?page=1&type=movie').text, "html5lib")
page_sub2 = BeautifulSoup(requests.get('https://studyenglishwords.com/browse?page=2&type=movie').text, "html5lib")

In [8]:
subs = page_sub1.find_all("a", {"class":"type-common type-movie"})+page_sub2.find_all("a", {"class":"type-common type-movie"})

In [9]:
sub_link = {}
for sub in subs:
    sub_link[re.sub('/', '-', sub.text.strip())] = sub.get('href')

In [12]:
 columns = ['en', 'ru']

for sub in sub_link:
    # открыть книгу
    bookpage = BeautifulSoup(requests.get(source+sub_link[sub]).text, "html5lib")
    df = pd.DataFrame(columns=columns)
    # пока есть страницы
    while bookpage:
        # сохраняем их в df
        save_page(bookpage, df, substitutions)
        # получаем следующую страницу
        next_page = bookpage.find("span", {"class":"next"}).find("a")
        if next_page:
            bookpage = BeautifulSoup(requests.get(source+sub_link[sub]+next_page.get("href")).text, "html5lib")
        else:
            bookpage = False
            
    # добавляем столбец со списком токенов
    df['ru_tokens'] = df['ru'].apply(tokenize)
    
    # сохранить датафрейм
    df.to_csv(sub.replace(' ', '_')+'.tsv', sep = '\t')
    print(sub+' is saved')
    
    del df
    gc.collect()

Храбрая сердцем - Brave (2012-06-21) is saved
Шрэк - Shrek (2001-05-16) is saved
Турист - The Tourist (2010-12-08) is saved
Хранители - Watchmen (2009-03-05) is saved
Убить Билла - Kill Bill: Vol. 1 (2003-10-10) is saved
Рататуй - Ratatouille (2007-06-22) is saved
Адвокат дьявола - The Devil's Advocate (1997-10-17) is saved
Тупой и еще тупее - Dumb and Dumber (1994-12-16) is saved
Леон - Léon (1994-09-14) is saved
Звёздные войны: Эпизод 1 - Скрытая угроза - Star Wars: Episode I - The Phantom Menace (1999-05-19) is saved
Пассажиры - Passengers (2016-12-21) is saved
Аватар - Avatar (2009-12-18) is saved
Сумерки. Сага. Затмение - The Twilight Saga: Eclipse (2010-06-23) is saved
Пятьдесят оттенков серого - Fifty Shades of Grey (2015-02-11) is saved
Голодные игры: Сойка-пересмешница. Часть I - The Hunger Games: Mockingjay - Part 1 (2014-11-18) is saved
Голодные игры: И вспыхнет пламя - The Hunger Games: Catching Fire (2013-11-15) is saved
Голодные игры - The Hunger Games (2012-03-12) is sav